<a href="https://colab.research.google.com/github/IshantWadhwa4/Neural_Networks_TensorFlow/blob/master/TensorFlow_Regration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
%tensorflow_version 2.x 
import tensorflow as tf

In [0]:
data_train = pd.read_csv(filepath_or_buffer='https://storage.googleapis.com/tf-datasets/titanic/train.csv')
data_test = pd.read_csv(filepath_or_buffer='https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [0]:
data_train_X = data_train.drop('survived',axis=1)
data_train_y = data_train['survived']

In [0]:
data_test_x = data_test.drop('survived',axis=1)
data_test_y = data_test['survived'] 

In [0]:
data_test_y.shape

(264,)

In [0]:
def get_numeric_data_columns(df):
  '''
      return list of all numeric data columns name
  '''
  return list(df._get_numeric_data().columns)


def get_catagorical_data_columns(df):
  '''
      return list of all catagoric data columns name
  '''
  return list(set(df.columns) - set(df._get_numeric_data().columns))

In [0]:
numeric_columns = get_numeric_data_columns(data_train_X)
catagorical_columns = get_catagorical_data_columns(data_train_X)

In [0]:
numeric_columns

['age', 'n_siblings_spouses', 'parch', 'fare']

In [0]:
catagorical_columns

['alone', 'sex', 'embark_town', 'deck', 'class']

In [0]:
# now we will create a feature column which has complete information about features
features_columns = []
for col in catagorical_columns:
  unique_variables = data_train_X[col].unique()
  features_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(col,unique_variables))

for feature_name in numeric_columns:
  features_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

  

In [0]:
tf_df = tf.data.Dataset.from_tensor_slices((dict(data_train_X),data_train_y))

In [0]:
tf_df.batch(32).repeat(100)

<RepeatDataset shapes: ({sex: (None,), age: (None,), n_siblings_spouses: (None,), parch: (None,), fare: (None,), class: (None,), deck: (None,), embark_town: (None,), alone: (None,)}, (None,)), types: ({sex: tf.string, age: tf.float64, n_siblings_spouses: tf.int64, parch: tf.int64, fare: tf.float64, class: tf.string, deck: tf.string, embark_town: tf.string, alone: tf.string}, tf.int64)>


In [0]:
data_train_X.shape

(627, 9)

In [0]:
# This function will convert pandas df to Tensor flow data frame for model traning
# data_df_x, --> Features Data frame
#label_df_y, --> Target variable data frame
#epochs --> How many time model should be feeded with same data for better understanding (more epochs overfit)
#shuffle=True --> shuffle the data every time
#batch_size=32 --> number of rows we input in single batch it is used when data size is large
def convert_pandasDF_to_tfDF(data_df_x,label_df_y,epochs=10,shuffle=True,batch_size=32):
  def tfdf_function():
    tf_df = tf.data.Dataset.from_tensor_slices((dict(data_df_x),label_df_y))
    if shuffle:
      tf_df.shuffle(1000)
    tf_df = tf_df.batch(batch_size=batch_size).repeat(epochs)
    return tf_df
  return tfdf_function 

train_input_fn = convert_pandasDF_to_tfDF(data_train_X,data_train_y)
test_input_fn = convert_pandasDF_to_tfDF(data_test_x,data_test_y,1,False)

In [0]:
## Create model
linear_model = tf.estimator.LinearClassifier(feature_columns = features_columns) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp1gt76ah8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
linear_model.train(train_input_fn)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorf

In [0]:
result = linear_model.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-06T04:36:53Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1gt76ah8/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.71983s
INFO:tensorflow:Finished evaluation at 2020-05-06-04:36:54
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.7651515, accuracy_

In [0]:
result

{'accuracy': 0.7651515,
 'accuracy_baseline': 0.625,
 'auc': 0.83966947,
 'auc_precision_recall': 0.7873041,
 'average_loss': 0.47249624,
 'global_step': 200,
 'label/mean': 0.375,
 'loss': 0.45887482,
 'precision': 0.69473684,
 'prediction/mean': 0.35994655,
 'recall': 0.6666667}

In [0]:
result = list(linear_model.predict(test_input_fn))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1gt76ah8/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
prob = []
for r in result:  
  prob.append(0 if r['probabilities'][0]>r['probabilities'][1] else 1)

In [0]:
data_test['prediction'] = prob

In [0]:
wrong_data = data_test[data_test['survived'] != data_test['prediction']]

In [0]:
wrong_data['survived'].value_counts()

1    33
0    29
Name: survived, dtype: int64